<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/big_bird_depresion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [17]:
notebook_login()

In [4]:
model_id = "christinacdl/bigbird_moderate_severe_depression"
##dataset_id = "ag_news"
# replace the value with your model: ex <hugging-face-user>/<model-name>
repository_id = "bigbird-depression-detection"

In [5]:
import pandas as pd
from datasets import ClassLabel, Dataset
data=pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
test=pd.read_csv('/content/DAIC_test_3sp.csv')
#label_test = ClassLabel(num_classes=2, names=[0, 1])

data.rename(columns = {'Group':'label'}, inplace = True)
test.rename(columns = {'Group':'label'}, inplace = True)

dataset = Dataset.from_dict(data)
test_dataset=Dataset.from_dict(test)

dataset = dataset.class_encode_column("label")
test_dataset = test_dataset.class_encode_column("label")

Stringifying the column:   0%|          | 0/10264 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10264 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1914 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [6]:
data['label'].value_counts()

0    5132
1    5132
Name: label, dtype: int64

In [7]:
type(dataset)

datasets.arrow_dataset.Dataset

In [8]:
#!pip install transformers[sentencepiece]

In [9]:

# Training and testing datasets
train_dataset = dataset
test_dataset = test_dataset
# Validation dataset
##val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    #print(tokenizer(batch["text"], padding=True, truncation=True, max_length=256)['input_ids'])
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
##val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [10]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [11]:
# Set dataset format
train_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
#val_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
test_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])

In [12]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [13]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features['label'].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 2
the labels: ['0', '1']


In [14]:
# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [15]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate


  
   learning_rate=2e-5,


In [18]:
import numpy as np
# Model
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config,ignore_mismatched_sizes=True)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)
from datasets import load_metric
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at christinacdl/bigbird_moderate_severe_depression and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cloning https://huggingface.co/Geee05/bigbird-depression-detection into local empty directory.


In [19]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Attention type 'block_sparse' is not possible if sequence_length: 242 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.664100,0.736353,0.631661,0.290030
2,0.593500,0.754283,0.620690,0.385787
3,0.591100,0.827020,0.597701,0.365733
4,0.553900,0.960976,0.603448,0.343993
5,0.349600,1.052859,0.613375,0.328494


<ipython-input-18-febfc10cb2a6>:29: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=6415, training_loss=0.5738013011816415, metrics={'train_runtime': 2896.2608, 'train_samples_per_second': 17.719, 'train_steps_per_second': 2.215, 'total_flos': 6426219801334080.0, 'train_loss': 0.5738013011816415, 'epoch': 5.0})

In [ ]:
# Evaluate the model
trainer.evaluate()